In [ ]:
import numpy as np
import torch

# courtesy of minhuan's code
def decom_inverse(G):
    eigvalues, eigvectors = np.linalg.eig(G)
    w = np.argsort(eigvalues)[::-1][:np.linalg.matrix_rank(G)]
    res = np.zeros(G.shape)
    for j in w:
        res += (1/eigvalues[j])*eigvectors[:,j].reshape(-1,1)@eigvectors[:,j].reshape(1,-1)
    return res

def NLL_gamma(data,mu,gamma,rank):
    reverse_det = 1./torch.sort(torch.eig(gamma,eigenvectors=True)[0][:,0], descending=True)[0][:rank]
    return 0.5*torch.sum(torch.log(reverse_det)) + 0.5*torch.trace(torch.mm(torch.mm(data-mu,gamma),(data-mu).T))/len(data)

def kronecker(A, B):
    return torch.einsum("ab,cd->acbd", A, B).view(A.size(0)*B.size(0),  A.size(1)*B.size(1))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# initialize the parameters
centroid = np.mean(cluster1,axis=0)
Cov_xx = np.cov(cluster1[:,:,0],rowvar=False)
Cov_yy = np.cov(cluster1[:,:,1],rowvar=False)
Cov_zz = np.cov(cluster1[:,:,2],rowvar=False)
gamma_0 = (decom_inverse(Cov_xx)+decom_inverse(Cov_yy)+decom_inverse(Cov_zz))/3.
L_0 = np.tril(gamma_0,k=-1) 
mask = np.tril(np.full((87,87),1),k=-1)
fixed = np.zeros((87,87))

## Tensorize 
LT = torch.tensor(L_0,dtype=torch.float64, requires_grad=True)
mu = torch.tensor(centroid.reshape(-1),dtype=torch.float64,device=device, requires_grad=True)
mask_ts = torch.tensor(mask, dtype=torch.float64, requires_grad=False)
fixed_ts = torch.tensor(fixed, dtype=torch.float64, requires_grad=False)
I3 = torch.eye(3,dtype=torch.float64, requires_grad=False)
data_tensor = torch.tensor(cluster1.reshape(-1,87*3),dtype=torch.float64)

# Optimize
mu_log=[]
gamma_log=[]
loss_log=[]
n_epoch=2000
batch_size=500
batches = int(len(data_tensor) / batch_size)
optimizer = torch.optim.Adam([mu,LT],lr=0.01)

for epoch in tqdm(range(n_epoch)):    
    indices = np.arange(len(data_tensor))
    np.random.shuffle(indices)
    
    for i in range(batches):
        
        if i == batches - 1:
            batch_indices = indices[i * batch_size:]
        else:
            batch_indices = indices[i * batch_size:(i + 1) * batch_size]
        
        optimizer.zero_grad()
        L = LT*mask_ts + (1-mask_ts)*fixed_ts
        gamma = kronecker(L + L.T - (L + L.T).sum(1).diag(), I3)
        loss = NLL_gamma(data_tensor[batch_indices],mu,gamma,258)
        mu_log.append(mu.detach().numpy().copy())
        gamma_log.append(gamma.detach().numpy().copy())
        loss_log.append(loss.detach().numpy().copy())
          
        loss.backward()
        optimizer.step()